In [9]:
# 📦 Import libraries
import pandas as pd
import os

# 📁 Path to your Excel files
DATA_DIR = 'Data'

# 📄 Load datasets
customers = pd.read_csv(os.path.join(DATA_DIR, 'olist_customers_dataset.csv'))
orders = pd.read_csv(os.path.join(DATA_DIR, 'olist_orders_dataset.csv'))
order_items = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_items_dataset.csv'))
order_payments = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_payments_dataset.csv'))
order_reviews = pd.read_csv(os.path.join(DATA_DIR, 'olist_order_reviews_dataset.csv'))
products = pd.read_csv(os.path.join(DATA_DIR, 'olist_products_dataset.csv'))
product_translation = pd.read_csv(os.path.join(DATA_DIR, 'product_category_name_translation.csv'))

# ✅ Preview some key tables
print("Orders:")
display(orders.head())

print("Order Items:")
display(order_items.head())

print("Payments:")
display(order_payments.head())


Orders:


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


Order Items:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


Payments:


,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [13]:
# Merge orders + payments
orders_payments = pd.merge(orders, order_payments, on='order_id', how='left')

# Merge with order_items
orders_items = pd.merge(orders_payments, order_items, on='order_id', how='left')

# Merge with reviews
orders_full = pd.merge(orders_items, order_reviews[['order_id', 'review_score']], on='order_id', how='left')

# Merge with products and product_category names
products_named = pd.merge(products, product_translation, on='product_category_name', how='left')
orders_full = pd.merge(orders_full, products_named[['product_id', 'product_category_name_english']], on='product_id', how='left')

# Merge with customers (to bring in state/city)
orders_full = pd.merge(orders_full, customers[['customer_id', 'customer_city', 'customer_state']], on='customer_id', how='left')

# 🔧 Create new features
orders_full['total_payment'] = orders_full['payment_value']
orders_full['shipping_days'] = (
    pd.to_datetime(orders_full['order_delivered_customer_date']) - 
    pd.to_datetime(orders_full['order_purchase_timestamp'])
).dt.days

orders_full['delivery_delay'] = (
    pd.to_datetime(orders_full['order_delivered_customer_date']) - 
    pd.to_datetime(orders_full['order_estimated_delivery_date'])
).dt.days

# Clean nulls in delivery_delay
orders_full['delivery_delay'] = orders_full['delivery_delay'].fillna(0)

# Preview
orders_full[['order_id', 'product_category_name_english', 'customer_state', 'review_score', 'total_payment', 'shipping_days', 'delivery_delay']].head()




,order_id,product_category_name_english,customer_state,review_score,total_payment,shipping_days,delivery_delay
0,e481f51cbdc54678b7cc49136f2d6af7,housewares,SP,4.0,18.12,8.0,-8.0
1,e481f51cbdc54678b7cc49136f2d6af7,housewares,SP,4.0,2.00,8.0,-8.0
2,e481f51cbdc54678b7cc49136f2d6af7,housewares,SP,4.0,18.59,8.0,-8.0
3,53cdb2fc8bc7dce0b6741e2150273451,perfumery,BA,4.0,141.46,13.0,-6.0
4,47770eb9100c2d0c44946d9cf07ec65d,auto,GO,5.0,179.12,9.0,-18.0


In [17]:
# 📤 Export cleaned dataset for Tableau
OUTPUT_DIR = '../output'  # Adjust if output folder is elsewhere

# Create folder if needed
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Select final columns for Tableau
final_df = orders_full[[
    'order_id',
    'product_category_name_english',
    'customer_state',
    'review_score',
    'total_payment',
    'shipping_days',
    'delivery_delay'
]]

# Save as CSV
output_path = os.path.join(OUTPUT_DIR, 'olist_cleaned_for_tableau.csv')
final_df.to_csv(output_path, index=False)

print("✅ Cleaned dataset saved to:", output_path)


✅ Cleaned dataset saved to: ../output\olist_cleaned_for_tableau.csv
